
# Vision and Cognitive Systems - Project


<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Vision_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries


## Setting up the working space

In this first cell we check if the notebook is runnig in Colab. In this case we need some additional work to set properly the environmet. We need also to mount our vision drive. In local machine instead we need to add the Geoestimation folder of our paper in the paths where python searches for libraries.

In [1]:
# with this line we can check if we are in colab or not
import os
import sys
from pathlib import Path
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir(cwd /'drive'/'MyDrive'/'GeoEstimation')
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✨🍰✨ Everything looks OK!


In [ ]:
# this cell takes a lot of time on colab!
import sys
in_colab = 'google.colab' in sys.modules
if in_colab:
    import condacolab
    condacolab.check()
    import os
    os.chdir(r'/content/drive/MyDrive/GeoEstimation')
    print(os.getcwd())
    !conda env update -n base -f environment.yml
    # The following is ridiculous, I know, but it seems to work
    !pip uninstall torchtext
    !pip install torchtext==0.7

In theory we need to install some specific packages with certain version to account for the original environment in which the paper results were obtained:
```
  - python=3.8
  - msgpack-python=1.0.0
  - pandas=1.1.5
  - yaml=0.2.5
  - tqdm=4.50
  - cudatoolkit=10.2
  - pytorch=1.6
  - torchvision=0.7
  - pytorch-lightning=1.0.1
  - pip
  - pip:
    - s2sphere==0.2.5
```

## A transfer learning example: the strenght of pytorch-lightning

Here we want to show in a nutshell the transfer learning approach from a pretrained model using both standard code and pytorch-lightning, to highlight the differences. Moreover we are going to load the same pretrained model (ResNet50) used by the authors as backbone to develop their ML model. For seek of semplicity we are going to re-train this model on the Cifar10. First let's see the classic torch approach:

In [1]:
#libraries
from torchvision import models
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import softmax, cross_entropy
from torch.optim import Adam
from torch.utils.data import random_split

#check for GPU
want_gpu = True
if want_gpu and torch.cuda.is_available():
    gpu = 1
else:
    gpu = None

#download the pretrained model
backbone = models.resnet50(pretrained = True)

#download and normalize the CIFAR10 dataset
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
cf10_transforms = transforms.Compose([
    transforms.ToTensor(),
    normalize
])
cifar_10 = CIFAR10('.',train=True, download = True, transform=cf10_transforms) 

'''#train, validation and test split (we have to set the seed)
train_data, val_data, test_data = random_split(cifar_10, [40000, 10000, 10000] )'''

#prepare the batches
train_loader = DataLoader(cifar_10, batch_size=32, shuffle=True)
'''val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)'''

# We add to the last layer with a fully connected one to match our number of classes (=10):
# We treat the outputs of resnet as high level features (we could use them with any classifier instead of a FC)
finetune_layer = torch.nn.Linear(backbone.fc.out_features, 10) 
#finetune_layer = torch.nn.Linear(backbone.fc.in_features, 10) is for REPLACE THE LAST LAYER

#define the optimizer
optimizer = Adam(finetune_layer.parameters(), lr = 1e-4)

#we set a limit for the number of batches in each epoch, since we are not interesting in training proper this model
limit_train_batches = 10

#training
for epoch in range(10):
    print(f'Epoch {epoch}')
    for i,batch in enumerate(train_loader):
      if i<limit_train_batches:

          x, y = batch
          #we do not waste memory recording the gradient on the backbone
          with torch.no_grad():
              #(b, 3, 32, 32) -> (b, 1000)
              features = backbone(x)

          # (b, 1000) -> (b, 10)
          preds = finetune_layer(features)
          loss = cross_entropy(preds, y)

          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
          print(loss.item())


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to .
Epoch 0
3.1930480003356934
3.123041868209839
3.0640506744384766
3.0658154487609863
3.264195442199707
3.7586264610290527
3.468670606613159
3.4459760189056396
3.164868116378784
3.2048826217651367
Epoch 1
3.4092202186584473
3.2618446350097656
3.543487787246704
2.9497785568237305
3.0474600791931152
3.175663709640503
2.6673035621643066
3.1997504234313965
2.9967129230499268
3.018550395965576
Epoch 2
3.0200114250183105
3.6485610008239746
2.8175318241119385
3.3879618644714355
2.957050323486328
2.942509889602661
3.0313799381256104
2.8381459712982178
2.9101810455322266
2.69409441947937
Epoch 3
3.0937843322753906
2.800339698791504
2.7473137378692627
2.8794608116149902
2.829761266708374
2.7380940914154053
2.991612434387207
2.689542770385742
2.536581516265869
3.066215991973877
Epoch 4
2.942972421646118
2.8620858192443848
2.899718999862671
2.8575029373168945
2.734931707382202
2.764251708984375
2.7208406925201416
2.7029600143432617
2.576611042022705
2.77293992

KeyboardInterrupt: ignored

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy

class ImageClassifier(pl.LightningModule):
    def __init__(self, num_classes=10 , lr = 1e-3):
        super().__init__()
        #this setting save as the time to define an attribute for each hyperparameter --> self.hparams.<parameter>
        self.save_hyperparameters() #Pytorch-lightning trick!
        self.backbone = models.resnet50(pretrained = True)
        self.finetune_layer = torch.nn.Linear(backbone.fc.out_features, num_classes)

    def training_step(self, batch, batch_idx): #these methods are standard methods in LightningModule
        x, y = batch

        #we decide whether to freeze the backbone or not on the base of the number of epochs
        if self.trainer.current_epoch < 10:
            with torch.no_grad():
                #(b, 3, 32, 32) -> (b, 1000)
                features = self.backbone(x)
        else:
            features = self.backbone(x)

        # (b, 1000) -> (b, 10)
        preds = self.finetune_layer(features)
        loss = cross_entropy(preds, y)
        #we don't need anymore loss.backward(), optimizer.step(), optimizer.zero_grad()
        self.log('train_loss', loss) # we will see later this method of LightningModule
        self.log('train_loss', accuracy(preds, y))
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch

        features = self.backbone(x)

        # (b, 1000) -> (b, 10)
        preds = self.finetune_layer(features)
        loss = cross_entropy(preds, y)
        #we don't need anymore loss.backward(), optimizer.step(), optimizer.zero_grad()
        self.log('val_loss', loss) # we will see later this method of LightningModule
        self.log('val_loss', accuracy(preds, y))
        return loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr =self.hparams.lr) 
        #we can safely pass all the parameters since in the backbone we are not computing the gradient
        return optimizer
        

At this point we have  very flexible object ```ImageClassifier```.

In [ ]:
from pl_bolts.datamodules import CIFAR10DataModule

#Bolts save us the time of train, vla, test split and using 3 different torch.Dataloader for each of them
dm = CIFAR10DataModule('.') 

classifier = ImageClassifier()
logger = pl.loggers.TensorBoardLogger(name = f'pretrained model 1', save_dir = 'lightning_logs')
trainer = pl.Trainer(
    max_epochs = 2, # set the number of epochs if <1000 (=defult)
    progress_bar_refresh_rate = 20, 
    logger = logger,
    #gpus=1, 
    limit_train_batches = 50,
    #limit_val_batches = 2,
    #check_val_every_n_epoch = 5
    #fast_dev_run=True # add this to have a fast chech of bugs
    ) 
trainer.fit(classifier, dm) #we can use the normal train_loader we defined previously



We can use this very nice [tool](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.loggers.tensorboard.html#module-pytorch_lightning.loggers.tensorboard) form Pytorch-Lighting

In [ ]:
# start tensorboard
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

### Self-supervised transfer learning with Lightning

PyTorch Lightning implementation of SwAV adapted from the [official implementation](https://arxiv.org/abs/2006.09882), whose authors used the same pretrained model (ResNet50 trained on ImageNet). We can simply import this model from Lightning-Bolt and define a class very similar to the previous one for our classifer. 

In [ ]:
from torchvision import models
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import softmax, cross_entropy
from torch.optim import Adam
from torch.utils.data import random_split
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import SwAV
from pl_bolts.datamodules import CIFAR10DataModule

#Bolts save us the time of train, vla, test split and using 3 different torch.Dataloader for each of them
dm = CIFAR10DataModule('.') 

#weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/bolts_swav_imagenet/swav_imagenet.ckpt'
#weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/swav/swav_imagenet/swav_imagenet.pth.tar'
swav = SwAV.load_from_checkpoint(r'C:\Users\latta\.cache\torch\hub\checkpoints\swav_imagenet.pth.tar', strict=False)

class SSLImageClassifier(pl.LightningModule):
    def __init__(self, num_classes=10 , lr = 1e-3):
        super().__init__()
        self.save_hyperparameters() 
        self.backbone = swav.model #model pretrained on ImageNet without labels
        self.finetune_layer = torch.nn.Linear(3000, num_classes)

    def training_step(self, batch, batch_idx): #these methods are standard methods in 
        x, y = batch

        #we decide whether to freeze the backbone or not on the base of the number of epochs
        if self.trainer.current_epoch < 10:
            with torch.no_grad():
                #(b, 3, 32, 32) -> (b, 1000)
                (f1, f2) = self.backbone(x)
                features = f2
        else:
            (f1, f2) = self.backbone(x)
            features = f2

        # (b, 1000) -> (b, 10)
        preds = self.finetune_layer(features)
        loss = cross_entropy(preds, y)
        #we don't need anymore loss.backward(), optimizer.step(), optimizer.zero_grad()
        self.log('train_loss', loss) # we will see later this method of LightningModule
        self.log('train_loss', accuracy(preds, y))
        return loss

    def validation_step(self, batch, batch_idx): #these methods are standard methods in 
        x, y = batch
        
        (f1, f2) = self.backbone(x)
        features = f2

        # (b, 1000) -> (b, 10)
        preds = self.finetune_layer(features)
        loss = cross_entropy(preds, y)
        #we don't need anymore loss.backward(), optimizer.step(), optimizer.zero_grad()
        self.log('val_loss', loss) # we will see later this method of LightningModule
        self.log('val_loss', accuracy(preds, y))
        return loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr =self.hparams.lr) 
        #we can safely pass all the parameters since in the backbone we are not computing the gradient
        return optimizer

'''backbone = models.resnet50(pretrained = True)
finetune_layer = torch.nn.Linear(backbone.fc.out_features, 10) 
#define the optimizer
optimizer = Adam(finetune_layer.parameters(), lr = 1e-4)
'''
ssl_classifier = SSLImageClassifier()
logger = pl.loggers.TensorBoardLogger(name = f'pretrained model 2 (self superised)', save_dir = 'lightning_logs')
trainer = pl.Trainer(
    max_epochs = 2,
    #progress_bar_refresh_rate = 20, 
    gpus=0, 
    limit_train_batches = 50#, 
    #fast_dev_run=True # add this to have a fast chech of bugs
    ) 
trainer.fit(ssl_classifier, dm) #we can use the normal train_loader we defined previously

In [ ]:
# start tensorboard
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

## Reproduce paper results


To begin we try to reproduce the paper results on their test set.

In [ ]:
from pathlib import Path
from math import ceil

import pandas as pd
import torch
import pytorch_lightning as pl

from classification.train_base import MultiPartitioningClassifier # class defining our model
from classification.dataset import FiveCropImageDataset # class for preparing the images before giving them to the NN

### Load the model

In [ ]:
# where model's params and hyperparams are saved
checkpoint = "models/base_M/epoch=014-val_loss=18.4833.ckpt"
hparams = "models/base_M/hparams.yaml"

In [ ]:
# load_from_checkpoint is a static method from pytorch lightning, inherited by MultiPartitioningClassifier
# it permits to load a model previously saved, in the form of a checkpoint file, and one with hyperparameters
# MultiPartitioningClassifier is the class defining our model
model = MultiPartitioningClassifier.load_from_checkpoint(
    checkpoint_path=checkpoint,
    hparams_file=hparams,
    map_location=None
)

In [ ]:
type(pl.LightningModule)

In [ ]:
#to allow GPU
want_gpu = True
if want_gpu and torch.cuda.is_available():
    gpu = 1
else:
    gpu = None

# the class Trainer from pythorch lightining is the one responsible for training a deep NN
# it can initialize the model, run forward and backward passes, optimize, print stats, early stop...
wanted_precision = 32 #16 for half precision (how many bits for each number)
trainer = pl.Trainer(gpus=gpu, precision=wanted_precision)

### Load and initialize the images

In [ ]:
# where images are saved
image_dir = "resources/images/im2gps"
meta_csv = "resources/images/im2gps_places365.csv"

In [ ]:
import pandas as pd
first_csv = pd.read_csv(meta_csv)

In [ ]:
#FiveCropImageDataset is the class for preparing the images before giving them to the NN
# in particular, it creates five different crops for every image
dataset = FiveCropImageDataset(meta_csv, image_dir)

In [ ]:
batch_size = 64
dataloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    shuffle=False,
                    num_workers=4 #number ot threads used for parallelism (cores of CPU?)
                )

### Run the model on the test set

In [ ]:
results = trainer.test(model, test_dataloaders=dataloader, verbose=False)

### Look at the results

In [ ]:
# formatting results into a pandas dataframe
df = pd.DataFrame(results[0]).T
#df["dataset"] = image_dir
df["partitioning"] = df.index
df["partitioning"] = df["partitioning"].apply(lambda x: x.split("/")[-1])
df.set_index(keys=["partitioning"], inplace=True) #keys=["dataset", "partitioning"] in case
print(df)

In [ ]:
# to save the dataframe on a csv file
fout = 'test_results.csv'
df.to_csv(fout)

In [ ]:
os.chdir(r'/content/drive/MyDrive/GeoEstimation/resources/images/im2gps')
print(len(os.listdir()))
os.chdir(r'/content/drive/MyDrive/GeoEstimation')
print(os.getcwd())
import torch
print(torch.cuda.is_available())

# Output would be True if Pytorch is using GPU otherwise it would be False.
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

In [ ]:
#@title
#libraries to import
#known
import pandas as pd
import numpy as np
import os
import re
import torchvision
import torch
import PIL
from PIL import Image
from PIL import ImageFile
import sys
import time
from math import ceil



#Unknown
from typing import Union
from io import BytesIO
import random
from argparse import Namespace, ArgumentParser
from pathlib import Path
from multiprocessing import Pool
from functools import partial
import requests
import logging
import json
import yaml
from tqdm.auto import tqdm
#from classification.train_base import MultiPartitioningClassifier
#from classification.dataset import FiveCropImageDataset

#to divide
from classification import utils_global
from classification.s2_utils import Partitioning, Hierarchy
from classification.dataset import MsgPackIterableDatasetMultiTargetWithDynLabels


The main link and paper that we need to follow is [this](https://github.com/TIBHannover/GeoEstimation) and [this](https://github.com/TIBHannover/GeoEstimation/releases/) for the pretrained models.

Davide ha trovato questo che forse è meglio [kaggle](https://www.kaggle.com/code/habedi/inspect-the-dataset/data)

## Our dataset

We have downloaded a new 10k dataset with gps coordinates. We need the labels for the scenes.


In [2]:
import numpy as np
import importlib
imported_module = importlib.import_module("scene_classification")
importlib.reload(imported_module)
import scene_classification
from scene_classification import SceneClassifier


In [ ]:
import pandas as pd

#initialiaze the classifier
scene_classifier = SceneClassifier(runtime='cpu')

#list of the images with full path
path = r'/content/drive/MyDrive/GeoEstimation/resources/images/new_data10k'
path_list = os.listdir(path)
path_list = [path+'/'+im for im in path_list if im[-3:]=='jpg']
print('num of images is ' , len(path_list))

#original file csv with images info
data_10k = pd.read_csv(r'/content/drive/MyDrive/GeoEstimation/resources/images/final_dataset_10k.csv', sep=';')
print('the origninal df of images info is')
display(data_10k.head())

#classification of the images, producing both S3_labels that probs
places_prob, S3_labels = scene_classifier.process_images(path_list,b_size=128)
print('num of triplette of probabilities is ' , len(places_prob))
print('num of  S3_labels is ', len(S3_labels))

#new file csv formation
new_data = pd.DataFrame(data = np.asarray(places_prob), columns=['Prob_indoor','Prob_natural','Prob_urban'])
S3_labels_data =  pd.DataFrame(data = np.asarray([S3_labels]).T ,columns=['S3_label'])
images_name = [im[:-4] for im in os.listdir(path) if im[-3:]=='jpg']
images_name_df = pd.DataFrame(data = images_name ,  columns=['photo_id'])

#new df with the new information
new_df = pd.concat([images_name_df,S3_labels_data,new_data],axis=1)
print('the new data column we have now are')
display(new_df.head(20))

#final merge with the original csv
new_df_full = data_10k.merge(right = new_df, on='photo_id')
print('final DataFrame')
new_df_full.head()

#save the new dataframe in a csv
print("Let's save the results")
new_df_full.to_csv(path_or_buf=r'/content/drive/MyDrive/GeoEstimation/resources/images/data10k_places365.csv', sep=',')


Loading scene hierarchy ...
num of images is  9446
the origninal df of images info is


,Unnamed: 0,photo_id,owner,gender,occupation,title,description,faves,lat,lon,u_city,u_country,taken,weather,season,daytime,base_url,url
0,0,17271526139,130418712@N05,1.0,NaN,Rio Trejo,Son numerosos los rios y arroyos que discurren...,701.0,"36,861544","-5,177747",NaN,NaN,2015-04-26 17:11:11,NaN,1.0,2.0,https://www.flickr.com/photos/130418712@N05/17...,https://live.staticflickr.com/65535/1727152613...
1,1,17776887679,55101137@N02,1.0,NaN,2015-05-13-022FD PH-XRD,<u><b>Aircraft Type - Registration - (c/n)</b>...,1.0,"51,463766","5,392935",Bodmin,United Kingdom,2015-05-13 00:00:22,9.0,1.0,3.0,https://www.flickr.com/photos/55101137@N02/177...,https://live.staticflickr.com/5335/17776887679...
2,2,17898331633,55101137@N02,1.0,NaN,2015-05-17-022FD OO-GWA,<u><b>Aircraft Type - Registration - (c/n)</b>...,2.0,"51,190492","4,453765",Bodmin,United Kingdom,2015-05-17 00:00:22,9.0,1.0,3.0,https://www.flickr.com/photos/55101137@N02/178...,https://live.staticflickr.com/525/17898331633_...
3,3,17940239919,55101137@N02,1.0,NaN,2015-05-14-020FD D-1553,<u><b>Aircraft Type - Registration - (c/n)</b>...,0.0,"51,326247","6,085953",Bodmin,United Kingdom,2015-05-14 00:00:20,9.0,1.0,3.0,https://www.flickr.com/photos/55101137@N02/179...,https://live.staticflickr.com/8860/17940239919...
4,4,17963122505,55101137@N02,1.0,NaN,2015-05-13-025FD EI-DLI,<u><b>Aircraft Type - Registration - (c/n)</b>...,2.0,"51,463766","5,392935",Bodmin,United Kingdom,2015-05-13 00:00:25,9.0,1.0,3.0,https://www.flickr.com/photos/55101137@N02/179...,https://live.staticflickr.com/5457/17963122505...


We have 74 batches
We are at batch  1
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  2
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  3
this batch has size  torch.Size([127, 3, 256, 256])
We are at batch  4
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  5
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  6
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  7
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  8
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  9
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  10
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  11
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  12
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  13
this batch has size  torch.Size([128, 3, 256, 256])
We are at batch  14
this batch has size  torch.Size([1